# Linear Programming

Linear programming is a mathematical technique to achieve the best outcome (such as maximum profit, or least possible cost), whilst having various constraints.

For our course, we will be using the `pulp` library which you can find more information __[here](https://www.coin-or.org/PuLP/pulp.html)__.

## Problem 3

A gold processor has two sources of gold ore, source A and source B. 

In order to keep his plant running, at least three tons of ore must be processed each day. Ore from source A costs $20 per ton to process, and ore from source B costs $10 per ton to process. Costs must be kept to less than $80 per day. 

Moreover, Federal Regulations require that the amount of ore from source B cannot exceed twice the amount of ore from source A. 

If ore from source A yields 2 oz. of gold per ton, and ore from source B yields 3 oz. of gold per ton, how many tons of ore from both sources must be processed each day to maximize the amount of gold extracted subject to the above constraints?

<br/>

## Setting up the Problem

The first task is to set up the problem by:
- Creating an `LpProblem`
- Defining the variables (including any bounds, and data type)

In [15]:
# import libraries
from pulp import *

# create the linear programming model
lp = LpProblem("Gold", LpMaximize)

# define variables
a = LpVariable(name='Source A', lowBound=0, cat='Integer')
b = LpVariable(name='Source B', lowBound=0, cat='Integer')

## Determine the Objective

Next step is to determine the objective

In this case we have: 
- Source A that has a yield of 2oz per tonne hence we have $2*a$
- Source B that has a yield of 3oz per tonne; hence we have $3*b$

We want to maximize the profit, so we have $max(p) = 2*a + 3*b$

This can be coded as following

In [16]:
lp.objective = (2*a) + (3*b)
lp

Gold:
MAXIMIZE
2*Source_A + 3*Source_B + 0
VARIABLES
0 <= Source_A Integer
0 <= Source_B Integer

## Setting up the Constraints

The next step is to plot out the constraints. In this case we have:
- minimum quota - At least 3 tonnes must be extracted per day, hence: $ a + b >= 3$
- cost - The total cost must be less than $80 daily. Source A costs $20 per tonne, and Source B costs $10 per tonne. The total cannot exceed the available budget hence: $20*a + 10*b <= 80$
- units -the amount of ore from source B cannot exceed twice the amount of ore from source A hence $2a - b >= 0$

More detail about the units constraint. Let's start with the basic premise that the amount extracted from Source B must be less than or equal to twice the amount of ore extracted from source A, hence:

$2a >= b$

Now we subtract $b$ from each side:

$2a - b >= 0$


In [17]:
lp.addConstraint(a + b >= 3, 'quota')
lp.addConstraint((20*a) + (10*b) <= 79, 'budget')
lp.addConstraint(b <= (2*a) , 'reg')
lp.constraints

OrderedDict([('quota', 1*Source_A + 1*Source_B + -3 >= 0),
             ('budget', 20*Source_A + 10*Source_B + -79 <= 0),
             ('reg', -2*Source_A + 1*Source_B + 0 <= 0)])

## Solving the Problem

The last step is to solve the problem. This can be done by calling the `solve` function.

In [18]:
# solve the linear programming problem
# 1 = Optimal, 2 = Infeasible, 3 = Unbounded, 4 = Undefined, 5 = Not Solved

status = lp.solve(PULP_CBC_CMD(msg=0))
print(status)

# print solution
for var in lp.variables():
    print(var,'=',value(var))
    
print('OPT',value(lp.objective))

1
Source_A = 2.0
Source_B = 3.0
OPT 13.0


If we look at the outcome we have:
- status = 1 - The staus is optimal (it can be not solved, infeasible, unbounded or undef)
- Source A = 2 tonnes
- Source B = 4 tonnes
- Maximum = 16 oz of gold
